In [2]:
import pandas as pd
import numpy as np
import Utils
import tensorflow as tf
import keras
import pickle
import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.utils import class_weight
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical, plot_model

import Utils.data_utils as data
import Utils.metric_utils as mtric
import getpass

In [3]:
path_train = 'dataset/Processed/Train_stop_coma_blind_num.pickle'
path_test = 'dataset/Processed/Test_stop_coma_blind_num.pickle'
path_w2v = 'W2V/wikipedia-pubmed-and-PMC-w2v.bin' # http://evexdb.org/pmresources/vec-space-models/
dataset = data.Dataset()
dataset.load_pickle(path_train, path_test, label_name='rating')
# dataset.list_to_stemmer('review') #snowball
# word_list = (x for sample in dataset.get_train().review for x in sample)

In [4]:
BoW = CountVectorizer(tokenizer=(lambda x: x), lowercase=False)
tfidf = TfidfTransformer()
BoW.fit(dataset.get_train().review)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function <lambda> at 0x7fc7d1d62730>, vocabulary=None)

In [5]:
Xtrain_bow = BoW.transform(dataset.get_train().review)
Xtest_bow = BoW.transform(dataset.get_test().review)

tfidf.fit(Xtrain_bow)
Xtrain_tfidf = tfidf.transform(Xtrain_bow)
Xtest_tfidf = tfidf.transform(Xtest_bow)

Ytrain = dataset.get_train().rating
Ytest = dataset.get_test().rating

In [12]:
model = SVC(kernel='linear', C=10, gamma='auto')
model.fit(Xtrain_tfidf, Ytrain)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [13]:
result = model.predict(Xtest_tfidf)
print(classification_report(Ytest, result, digits=4))

              precision    recall  f1-score   support

         1.0     0.6049    0.7653    0.6757      7299
         2.0     0.4728    0.4066    0.4372      2334
         3.0     0.4732    0.4041    0.4359      2205
         4.0     0.4508    0.3671    0.4047      1659
         5.0     0.4383    0.3889    0.4121      2710
         6.0     0.4485    0.3492    0.3927      2119
         7.0     0.4120    0.3326    0.3681      3091
         8.0     0.4409    0.4004    0.4197      6156
         9.0     0.4878    0.4006    0.4399      9177
        10.0     0.6493    0.7484    0.6953     17016

   micro avg     0.5530    0.5530    0.5530     53766
   macro avg     0.4878    0.4563    0.4681     53766
weighted avg     0.5386    0.5530    0.5417     53766



In [10]:
model2 = LinearSVC(C=10)
model2.fit(Xtrain_tfidf, Ytrain)

LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [11]:
result2 = model2.predict(Xtest_tfidf)
print(classification_report(Ytest, result2, digits=4))

              precision    recall  f1-score   support

         1.0     0.6064    0.7261    0.6609      7299
         2.0     0.4865    0.3710    0.4210      2334
         3.0     0.4799    0.3633    0.4135      2205
         4.0     0.4459    0.3454    0.3893      1659
         5.0     0.4361    0.3399    0.3820      2710
         6.0     0.4508    0.3285    0.3800      2119
         7.0     0.4350    0.3180    0.3674      3091
         8.0     0.4321    0.3611    0.3934      6156
         9.0     0.4692    0.4015    0.4327      9177
        10.0     0.6167    0.7748    0.6868     17016

   micro avg     0.5437    0.5437    0.5437     53766
   macro avg     0.4859    0.4330    0.4527     53766
weighted avg     0.5264    0.5437    0.5286     53766



In [11]:
tuned_parameters = [{'C': [1, 10, 100]}]

In [ ]:
clf = GridSearchCV(LinearSVC(), tuned_parameters, cv=5, n_jobs=-1)
clf.fit(Xtrain_tfidf, Ytrain)

In [16]:
result_grid = clf.predict(Xtest_tfidf)
print(classification_report(Ytest, result_grid, digits=4))

              precision    recall  f1-score   support

         1.0     0.5337    0.7200    0.6130      7299
         2.0     0.4197    0.2464    0.3105      2334
         3.0     0.3991    0.2331    0.2943      2205
         4.0     0.3791    0.2164    0.2755      1659
         5.0     0.3748    0.2391    0.2920      2710
         6.0     0.4070    0.2147    0.2811      2119
         7.0     0.3805    0.2255    0.2832      3091
         8.0     0.3716    0.2953    0.3291      6156
         9.0     0.4127    0.3328    0.3685      9177
        10.0     0.5701    0.7821    0.6595     17016

   micro avg     0.4963    0.4963    0.4963     53766
   macro avg     0.4248    0.3505    0.3707     53766
weighted avg     0.4690    0.4963    0.4686     53766

